
# Creating dataframe from csv filename should be passed from widgets. try if you can give from pipleine base parameter

In [0]:
from pyspark.sql.functions import*

In [0]:
filename=dbutils.widgets.text("Filename","")


In [0]:
filename=dbutils.widgets.get("ADF_input")
df_file=spark.read.csv(filename, header=True, sep=',')

#Check the dataframe created

In [0]:
df_file.show(10,False)

#Filter the data to find the values where the ArrDelay and DeptDelay are greater than zero because time cannot be in negative.

In [0]:
df_file.dtypes

In [0]:
df_slected=df_file.select("Year","Month","DayofMonth","DayOfWeek","DepTime","DepDelay","ArrTime","ArrDelay","Dest","Origin")
df_slected.show()

In [0]:
df_filt=df_slected.filter((col("DepDelay")>0) & (col("ArrDelay")>0))
df_filt.show()

# Apply 'groupby' on Destination and summarize by mean on arrival delay.

# Sort it in a descending order to identify the airport with the highest arrival delay.

In [0]:
from pyspark.sql.functions import col, desc

df_grouped = df_filt.groupby("Dest").agg(mean("ArrDelay").alias("MeanArrdelay")).sort(col("MeanArrdelay"),ascending=False)
df_grouped.show()

In [0]:
from datetime import datetime
date=datetime.now()
display(date)

# write dataframe into adls entity/yyy/mm/dd and delta tables

In [0]:
year=datetime.now().strftime("%Y")
month=datetime.now().strftime("%m")
day=datetime.now().strftime("%d")
date=datetime.now().strftime("%D")
display(day)

In [0]:
df_grouped.write.format("csv").option("header", True).mode("overwrite").save('/mnt/adlstestcontainer/Test/AirlinesData/' + year + '/' + month + '/' + day)

In [0]:
df_grouped.write.format("delta").saveAsTable("sampledb.AirlinesData"+year,mode="overwrite")

#Your Observation based on analysis can you tell me if airport has highest arrival or departure delay.Is action needed?

In [0]:
df_grouped.show()

In [0]:
#yes